<a href="https://colab.research.google.com/github/bhattacharya5/NLP/blob/main/Assignment2_NLU_KnowledgeNet_Final_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Build a relation classifier that can detect a predefined class of relations, as specified in the dataset. 
#2. Create a subset of the KnowledgeNet data using sentences which contain any of the following relations: (make a subset of train.json with these relations only)
DATE_OF_BIRTH (PER–DATE)
RESIDENCE (PER–LOC) 
BIRTHPLACE (PER–LOC)
NATIONALITY (PER–LOC)
EMPLOYEE_OF (PER–ORG) 
EDUCATED_AT (PER–ORG) 

In [ ]:
import numpy as np
import pandas as pd
import string
import re
import sklearn
from tqdm import tqdm
import json

#nltk
import nltk
from textblob import Word 
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')

#Keras
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
#from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Embedding, SimpleRNN
from keras.models import Sequential
from keras.layers import LSTM, Dropout, Flatten, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, Conv1D

#Tensorflow
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

#sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import sklearn.feature_extraction.text as text
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Version:  2.12.0
Eager mode:  True
Hub version:  0.13.0
GPU is NOT AVAILABLE


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Read the JSON file into a DataFrame, with lines=True.
df = pd.read_json('https://raw.githubusercontent.com/diffbot/knowledge-net/master/dataset/train.json', lines=True)

# Print the first five rows of the DataFrame.
df.head(5)

,fold,source,documentId,documentText,passages
0,3,Diffbot Knowledge Graph,13205,Documents Assist\n\nFlorida Cheap Divorce - Si...,"[{'passageId': '13205:453:779', 'exhaustivelyA..."
1,2,Diffbot Knowledge Graph,7,Tata Chemicals Europe\n\nTata Chemicals Europe...,"[{'passageId': '7:23:206', 'exhaustivelyAnnota..."
2,1,Diffbot Knowledge Graph,29726,Morphine\n\nMorphine was an American alternati...,"[{'passageId': '29726:151:315', 'exhaustivelyA..."
3,4,Diffbot Knowledge Graph,13115,Norman Nadorff\n\nNorman J. Nadorff is an inte...,"[{'passageId': '13115:1833:1922', 'exhaustivel..."
4,2,DBpedia Abstract,7109,Abdullah bin Mohammed bin Saud Al Thani\n\nH.E...,"[{'passageId': '7109:41:205', 'exhaustivelyAnn..."


passge text - entities<br>
Property name - label

In [ ]:
# Extract the entities and relations from the DataFrame.
entities = df['passages'].values.tolist()
entities

[[{'passageId': '13205:453:779',
   'exhaustivelyAnnotatedProperties': [{'propertyId': '9',
     'propertyName': 'EDUCATED_AT',
     'propertyDescription': 'Describes the relationship between a person and the institution where she/he studied.'}],
   'passageStart': 453,
   'passageEnd': 779,
   'passageText': 'When she founded Apex Legal Document Preparation Services she became a Premium Member of the FALDP (Florida Association of Legal Document Preparers) and educated herself on the different types of divorce been offered across the State of Florida and learned from the best in the industry of legal document preparation services.',
   'facts': []}],
 [{'passageId': '7:23:206',
   'exhaustivelyAnnotatedProperties': [{'propertyId': '1',
     'propertyName': 'SUBSIDIARY_OF',
     'propertyDescription': 'Describes the relationship between a parent company and one of its subsidiaries.'}],
   'passageStart': 23,
   'passageEnd': 206,
   'passageText': 'Tata Chemicals Europe (formerly Brunne

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%cd "/content/gdrive/My Drive/Colab Notebooks/semester3/NLP/KnowledgeNet"

/content/gdrive/My Drive/Colab Notebooks/semester3/NLP/KnowledgeNet


In [ ]:
import io
from io import StringIO
import json
import os
data_file = '/content/gdrive/My Drive/Colab Notebooks/semester3/NLP/KnowledgeNet/train.json'
#data_file = 'train.json'

# Filter the data to include only sentences that contain the desired relations.
documents = {}
properties = {}
passagesWithError = set()
entities = []
relations = []
reader = io.open(data_file, "r")
for line in reader:
    document = json.loads(line)
    documentId = str(document["documentId"])
    documentText = document["documentText"]
    fold = document["fold"]
  
    source = "TREx"
    if "source" in document:
        source = document["source"]

    passages = []
    for passage in document["passages"]:
        passageId = passage["passageId"]
        passageStart = passage["passageStart"]
        passageEnd = passage["passageEnd"]
        passageText = passage["passageText"]
        
        # read all the annotated properties on the passage
        exhaustivelyAnnotatedProperties = []
        for property in passage["exhaustivelyAnnotatedProperties"]:
            propertyId = property["propertyId"]
            propertyName = property["propertyName"]
            propertyDescription = property["propertyDescription"]
            properties[propertyId] = propertyName
            for relation in ['DATE_OF_BIRTH', 'RESIDENCE', 'BIRTHPLACE', 'NATIONALITY', 'EMPLOYEE_OF', 'EDUCATED_AT']:
              if relation in propertyName:
                entities.append(passageText)
                relations.append(propertyName)

In [ ]:
entities[1]

'He is a member of Phi Beta Kappa and was a Fulbright Fellow in Law in Brazil (1980-1981).'

In [ ]:
relations[1]

'NATIONALITY'

In [ ]:
from sklearn.model_selection import train_test_split

# Create a training and testing set.
X_train, X_test, y_train, y_test = train_test_split(entities, relations, test_size=0.2)

In [ ]:
# Define the sequence lengths, max number of words and embedding dimensions
# Sequence length of each sentence. If more, truncate. If less, pad with zeros
MAX_SEQUENCE_LENGTH = 300
# Top 20000 frequently occurring words
MAX_NB_WORDS = 20000

# Get the frequently occurring words
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(entities)
train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

# dictionary containing words and their index
word_index = tokenizer.word_index
#vocabulary size
vocab_size = len(word_index)+1

# total words in the corpus
print('Found %s unique tokens.' % len(word_index))
# get only the top frequent words on train
train_data = tf.keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
# get only the top frequent words on test
test_data = tf.keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print(train_data.shape)
print(test_data.shape)

Found 10087 unique tokens.
(1779, 300)
(445, 300)


In [ ]:
train_labels = y_train
test_labels = y_test

#import library
from sklearn.preprocessing import LabelEncoder
# converts the character array to numeric array. Assigns levels to unique labels.
le = LabelEncoder()
le.fit(train_labels)
train_labels = le.transform(train_labels)
test_labels = le.transform(test_labels)
print(le.classes_)
print(np.unique(train_labels, return_counts=True))
print(np.unique(test_labels, return_counts=True))

LabelEncoder()

['DATE_OF_BIRTH' 'EDUCATED_AT' 'NATIONALITY' 'PLACE_OF_RESIDENCE']
(array([0, 1, 2, 3]), array([459, 301, 497, 522]))
(array([0, 1, 2, 3]), array([128,  72, 131, 114]))


In [ ]:
# changing data types
labels_train = to_categorical(np.asarray(train_labels))
labels_test = to_categorical(np.asarray(test_labels))
print('Shape of train data tensor:', train_data.shape)
print('Shape of label tensor:', labels_train.shape)
print('Shape of test data tensor :', test_data.shape)
print('Shape of label tensor:', labels_test.shape)

EMBEDDING_DIM = 100

Shape of train data tensor: (1779, 300)
Shape of label tensor: (1779, 4)
Shape of test data tensor : (445, 300)
Shape of label tensor: (445, 4)


### Implementing LSTM Model

In [ ]:
embid_dim = 300

#model training
print('Training LSTM model.')
model1 = Sequential()
model1.add(Embedding(MAX_NB_WORDS, embid_dim, input_length=MAX_SEQUENCE_LENGTH))
model1.add(LSTM(16, activation='relu', return_sequences=True))
model1.add(Dropout(0.2))
model1.add(BatchNormalization())
model1.add(Flatten())
model1.add(Dense(4,activation='softmax'))
model1.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
model1.fit(train_data, labels_train,  batch_size=16,  epochs=10,  validation_data=(test_data, labels_test))

Training LSTM model.
Epoch 1/10
112/112 [==============================] - 28s 230ms/step - loss: 0.5308 - accuracy: 0.3777 - val_loss: 0.6273 - val_accuracy: 0.3955
Epoch 2/10
112/112 [==============================] - 25s 225ms/step - loss: 0.3398 - accuracy: 0.7066 - val_loss: 0.6176 - val_accuracy: 0.4719
Epoch 3/10
112/112 [==============================] - 25s 225ms/step - loss: 0.2193 - accuracy: 0.8291 - val_loss: 0.5749 - val_accuracy: 0.4360
Epoch 4/10
112/112 [==============================] - 24s 218ms/step - loss: 0.1571 - accuracy: 0.8606 - val_loss: 0.5443 - val_accuracy: 0.4607
Epoch 5/10
112/112 [==============================] - 25s 224ms/step - loss: 0.1348 - accuracy: 0.8634 - val_loss: 0.6137 - val_accuracy: 0.4449
Epoch 6/10
112/112 [==============================] - 25s 225ms/step - loss: 0.1221 - accuracy: 0.8657 - val_loss: 0.7360 - val_accuracy: 0.4539
Epoch 7/10
112/112 [==============================] - 26s 231ms/step - loss: 0.1144 - accuracy: 0.8685 - val_

# 3.Create a Knowledge Graph that can store the information contained in these sentences. You can use any open-source graph database for this purpose. 

Here are the steps involved in creating a Knowledge Graph that can store the information contained in the sentences:

1. Create Neo4j graph database.

2. Create a schema for the Knowledge Graph. The schema defines the structure of the Knowledge Graph. It specifies the types of nodes and edges that are allowed in the Knowledge Graph.

3. Load the data into the Knowledge Graph. The data can be loaded into the Knowledge Graph using a variety of methods, such as Cypher queries, LOAD CSV statements, and Bulk API calls.

4. Query the Knowledge Graph. The Knowledge Graph can be queried using a variety of methods, such as Cypher queries, SPARQL queries, and GraphQL queries.

5. Visualize the Knowledge Graph. The Knowledge Graph can be visualized using a variety of tools, such as Neo4j Bloom, ArangoDB Graph Studio, and OrientDB Graph Inspector.

In [ ]:
!pip install py2neo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.0/177.0 kB 5.7 MB/s eta 0:00:00


In [ ]:
!pip install neo4j
!pip install neo4j_bloom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.5/187.5 kB 5.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.8.0-py3-none-any.whl size=258393 sha256=577fdb8d69704625d8370634b1db6799243698daeec0538f7fbfca013a2c2985
  Stored in directory: /root/.cache/pip/wheels/6b/b5/da/73f634944e04e625954d101cb175ac1aeb9b29751a37d3383e
Successfully built neo4j
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement neo4j_bloom (from versions: none)
ERROR: No matching distribution found for neo4j_bloom


In [ ]:
!pip install  neo4j-driver

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.6/187.6 kB 5.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j-driver: filename=neo4j_driver-5.8.0-py3-none-any.whl size=258592 sha256=b0992cc82cb79b54924c9c2779c1067d26594dc5325b5e9165c24a6ad8bedb33
  Stored in directory: /root/.cache/pip/wheels/43/0d/a3/662c716aaee0215231cecf86fd26a9ca459f5d72732b2b1860
Successfully built neo4j-driver


In [ ]:
import py2neo
import json

# Connect to the Neo4j database.
graph = py2neo.Graph("bolt://3.87.232.188:7687", auth=("neo4j", "lists-settlements-wardroom"))

# Create a schema for the Knowledge Graph.
#graph.run("CREATE CONSTRAINT FOR (p:Person) REQUIRE p.name IS UNIQUE")

data_file = '/content/gdrive/My Drive/Colab Notebooks/semester3/NLP/KnowledgeNet/train.json'

# Read the JSON file and extract properties and relations.
with open(data_file, 'r') as f:
    for line in f:
        document = json.loads(line)
        for passage in document["passages"]:
            for property in passage["exhaustivelyAnnotatedProperties"]:
                propertyName = property["propertyName"]
                for relation in ['DATE_OF_BIRTH', 'RESIDENCE', 'BIRTHPLACE', 'NATIONALITY', 'EMPLOYEE_OF', 'EDUCATED_AT']:
                    if relation in propertyName:                      
                      subject = property['propertyName']
                      obj = property['propertyDescription']
                      if relation == 'DATE_OF_BIRTH':
                          graph.create(py2neo.Node('Person', name=subject, birth_date=obj))
                      elif relation == 'RESIDENCE':
                          graph.create(py2neo.Node('Person', name=subject, residence=obj))
                      elif relation == 'BIRTHPLACE':
                          graph.create(py2neo.Node('Person', name=subject, birthplace=obj))
                      elif relation == 'NATIONALITY':
                          graph.create(py2neo.Node('Person', name=subject, nationality=obj))
                      elif relation == 'EMPLOYEE_OF':
                          graph.create(py2neo.Node('Person', name=subject, employer=obj))
                      elif relation == 'EDUCATED_AT':
                          graph.create(py2neo.Node('Person', name=subject, university=obj))

# Query the Knowledge Graph.
# Find all people who were born after a specific date.
query = '''
MATCH (p:Person)
WHERE p.birth_date > '1776-07-04'
RETURN p.name
'''
results = graph.run(query)
for result in results:
    print(result['p.name'])


In [55]:
import json
import requests
from neo4j import GraphDatabase

# Connect to the Neo4j database
uri = "bolt://3.87.232.188:7687"  
username = "neo4j"     
password = "lists-settlements-wardroom"    

driver = GraphDatabase.driver(uri, auth=(username, password))

# Read the JSON file line by line
json_url = "https://raw.githubusercontent.com/diffbot/knowledge-net/master/dataset/train.json"
response = requests.get(json_url, stream=True)

# Establish relationships in the graph database
with driver.session() as session:
    for line in response.iter_lines():
        if line:
            data = json.loads(line)

            person = data.get("person")
            birthplace = data.get("birthplace")
            residence = data.get("residence")
            birth_date = data.get("birth_date")

            if person:
                # Create or merge the person node
                session.run(
                    "MERGE (p:Person {name: $name})",
                    name=person
                )

                if birthplace:
                    # Create or merge the birthplace node
                    session.run(
                        "MERGE (b:Location {name: $name})",
                        name=birthplace
                    )

                    # Create the relationship between person and birthplace
                    session.run(
                        "MATCH (p:Person {name: $person}), (b:Location {name: $birthplace}) "
                        "MERGE (p)-[:BORN_IN]->(b)",
                        person=person, birthplace=birthplace
                    )

                if residence:
                    # Create or merge the residence node
                    session.run(
                        "MERGE (r:Location {name: $name})",
                        name=residence
                    )

                    # Create the relationship between person and residence
                    session.run(
                        "MATCH (p:Person {name: $person}), (r:Location {name: $residence}) "
                        "MERGE (p)-[:RESIDES_IN]->(r)",
                        person=person, residence=residence
                    )

                if birth_date:
                    # Create the birth date property for the person
                    session.run(
                        "MATCH (p:Person {name: $person}) "
                        "SET p.birth_date = $birth_date",
                        person=person, birth_date=birth_date
                    )

# Close the database connection
driver.close()


### 4. Connect the Knowledge Graph to a front end that can take in Natural Language Queries and give the answers back. You can use any open-source chatbot for this purpose. That way, the system will also have the power to continue a conversation rather than only Question-Answering.

In [45]:
!pip install -q neo4j-driver
!pip install -q gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.9/287.9 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.3/75.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.2/137.2 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:0

In [46]:
from neo4j import GraphDatabase
import spacy
import gradio as gr     

In [47]:
driver = GraphDatabase.driver("bolt://3.87.232.188:7687", auth=("neo4j", "lists-settlements-wardroom"))

In [48]:
nlp = spacy.load("en_core_web_sm")

In [50]:
def extract_entity(question, entity_type):
    doc = nlp(question)
    for ent in doc.ents:
        if ent.label_ in entity_type:
            return ent.text
    return None

In [51]:
def chatbot(input, history=[]):
    output = get_answer(input)
    history.append((input, output))
    return history, history

In [49]:
def get_answer(question):
    doc = nlp(question)
    person_name = extract_entity(question, ["PERSON"])
    if person_name is None:
        return "Sorry, I don't understand who you're asking about."
    
    with graph.session() as session:
        if "birth date" in question or "date of birth" in question:
            result = session.run(f"MATCH (p:Person) WHERE p.name = '{person_name}' RETURN p.birth_date")
            if result.peek() is None:
                return f"No information found for the person '{person_name}'"
            for record in result:
                return f"The birth date of '{person_name}' is: {record['p.birth_date']}"
        elif "residence" in question or "live" in question:
            result = session.run(f"MATCH (p:Person) WHERE p.name = '{person_name}' RETURN p.residence")
            if result.peek() is None:
                return f"No information found for the person '{person_name}'"
            for record in result:
                return f"The residence of '{person_name}' is: {record['p.residence']}"
        elif "birthplace" in question or "born" in question:
            result = session.run(f"MATCH (p:Person) WHERE p.name = '{person_name}' RETURN p.birthplace")
            if result.peek() is None:
                return f"No information found for the person '{person_name}'"
            for record in result:
                return f"The birthplace of '{person_name}' is: {record['p.birthplace']}"
        else:
            return "Sorry, I don't understand what you're asking."


In [57]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Keyboard interruption in main thread... closing server.
